In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from gensim.models import KeyedVectors
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.utils import to_categorical
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch
import time
from tensorflow.keras.callbacks import EarlyStopping
!pip install evaluate
import evaluate
import pyarrow
import requests
from transformers import Trainer
from transformers.trainer_utils import EvaluationStrategy
from transformers.training_args import TrainingArguments
!pip install peft
import peft
from peft import LoraConfig, get_peft_model, TaskType
!pip install bitsandbytes accelerate transformers
!pip install gdown

In [ ]:
#Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')
import gdown

# Google Drive file ID from the link
file_id = '14v5DE_jYcYhIAyzLUsA06fvnHGzGbO9Q'

# URL to be used with gdown
gdown_url = f'https://drive.google.com/uc?id={file_id}'

# Download the file
gdown.download(gdown_url, 'data.csv', quiet=False)

# Read the CSV file into a DataFrame
data = pd.read_csv('data.csv', encoding='utf-8')
data.head()
data.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Downloading...
From (original): https://drive.google.com/uc?id=14v5DE_jYcYhIAyzLUsA06fvnHGzGbO9Q
From (redirected): https://drive.google.com/uc?id=14v5DE_jYcYhIAyzLUsA06fvnHGzGbO9Q&confirm=t&uuid=07747acf-0fcf-4323-8e17-25f72d065acd
To: /content/data.csv
100%|██████████| 266M/266M [00:01<00:00, 180MB/s]


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [ ]:
# Get unique values
unique_values = data['primaryCategories'].unique()

print(unique_values)

['Health & Beauty' 'Electronics' 'Office Supplies'
 'Animals & Pet Supplies' 'Home & Garden' 'Electronics,Furniture'
 'Toys & Games,Electronics' 'Electronics,Media'
 'Office Supplies,Electronics']


In [ ]:
#Build new data frame out of selected columns
data = data[["dateAdded", "primaryCategories", "reviews.username", "brand", "reviews.text", "reviews.title", "reviews.rating"]]

data.head()

,dateAdded,primaryCategories,reviews.username,brand,reviews.text,reviews.title,reviews.rating
0,2015-10-30T08:59:32Z,Health & Beauty,Byger yang,Amazonbasics,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,3
1,2015-10-30T08:59:32Z,Health & Beauty,ByMG,Amazonbasics,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,4
2,2015-10-30T08:59:32Z,Health & Beauty,BySharon Lambert,Amazonbasics,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,5
3,2015-10-30T08:59:32Z,Health & Beauty,Bymark sexson,Amazonbasics,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,5
4,2015-10-30T08:59:32Z,Health & Beauty,Bylinda,Amazonbasics,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,5


In [ ]:
# Concatenate
data['reviews.text'] = data['reviews.text'] + ' ' + data['reviews.title']

data.head()

,dateAdded,primaryCategories,reviews.username,brand,reviews.text,reviews.title,reviews.rating
0,2015-10-30T08:59:32Z,Health & Beauty,Byger yang,Amazonbasics,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,3
1,2015-10-30T08:59:32Z,Health & Beauty,ByMG,Amazonbasics,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,4
2,2015-10-30T08:59:32Z,Health & Beauty,BySharon Lambert,Amazonbasics,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,5
3,2015-10-30T08:59:32Z,Health & Beauty,Bymark sexson,Amazonbasics,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,5
4,2015-10-30T08:59:32Z,Health & Beauty,Bylinda,Amazonbasics,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,5


In [ ]:
#Define 'data' variable
data = pd.DataFrame(data)

# Convert the 'dateAdded' column to datetime format
data['dateAdded'] = pd.to_datetime(data['dateAdded'])

# Extract the year and create a new column 'year'
data['year'] = data['dateAdded'].dt.year

# Drop column
data.drop(['dateAdded'], axis=1, inplace=True)

# Display the resulting DataFrame
data.head()

,primaryCategories,reviews.username,brand,reviews.text,reviews.title,reviews.rating,year
0,Health & Beauty,Byger yang,Amazonbasics,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,3,2015
1,Health & Beauty,ByMG,Amazonbasics,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,4,2015
2,Health & Beauty,BySharon Lambert,Amazonbasics,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,5,2015
3,Health & Beauty,Bymark sexson,Amazonbasics,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,5,2015
4,Health & Beauty,Bylinda,Amazonbasics,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,5,2015


In [ ]:
# Check for missing values in the entire DataFrame
missing_values = data.isnull().sum()

# Display the count of missing values for each column
print(missing_values)

primaryCategories    0
reviews.username     5
brand                0
reviews.text         0
reviews.title        0
reviews.rating       0
year                 0
dtype: int64


In [ ]:
# Remove rows where 'reviews.username' is empty
data = data.dropna(subset=['reviews.username'])

# Check for missing values in the entire DataFrame
missing_values = data.isnull().sum()

# Display the count of missing values for each column
print (missing_values)
data.head()

primaryCategories    0
reviews.username     0
brand                0
reviews.text         0
reviews.title        0
reviews.rating       0
year                 0
dtype: int64


,primaryCategories,reviews.username,brand,reviews.text,reviews.title,reviews.rating,year
0,Health & Beauty,Byger yang,Amazonbasics,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,3,2015
1,Health & Beauty,ByMG,Amazonbasics,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,4,2015
2,Health & Beauty,BySharon Lambert,Amazonbasics,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,5,2015
3,Health & Beauty,Bymark sexson,Amazonbasics,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,5,2015
4,Health & Beauty,Bylinda,Amazonbasics,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,5,2015


In [ ]:
# Get the value counts for the 'reviews.rating' column
rating_counts = data['reviews.rating'].value_counts()
print(rating_counts)

reviews.rating
5    19893
4     5648
3     1206
1      965
2      615
Name: count, dtype: int64


In [ ]:
# Function to determine negative, neutral and positive
def review_sentiment(rating):
    if rating in [1, 2]:
        return "negative"
    elif rating in [3]:
        return "neutral"
    elif rating in [4, 5]:
        return "positive"
    else:
        return None

In [ ]:
# Apply the reviews rating
data['sentiment'] = data['reviews.rating'].apply(review_sentiment)
data.head()

,primaryCategories,reviews.username,brand,reviews.text,reviews.title,reviews.rating,year,sentiment
0,Health & Beauty,Byger yang,Amazonbasics,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,3,2015,neutral
1,Health & Beauty,ByMG,Amazonbasics,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,4,2015,positive
2,Health & Beauty,BySharon Lambert,Amazonbasics,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,5,2015,positive
3,Health & Beauty,Bymark sexson,Amazonbasics,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,5,2015,positive
4,Health & Beauty,Bylinda,Amazonbasics,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,5,2015,positive


In [ ]:
#Drop the reviews.rating column, since we are using sentiment.
data.drop(['reviews.rating'], axis=1, inplace=True)
data.head()

,primaryCategories,reviews.username,brand,reviews.text,reviews.title,year,sentiment
0,Health & Beauty,Byger yang,Amazonbasics,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,2015,neutral
1,Health & Beauty,ByMG,Amazonbasics,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,2015,positive
2,Health & Beauty,BySharon Lambert,Amazonbasics,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,2015,positive
3,Health & Beauty,Bymark sexson,Amazonbasics,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,2015,positive
4,Health & Beauty,Bylinda,Amazonbasics,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,2015,positive


In [ ]:
# Import dataset tools
from datasets import Dataset, DatasetDict

# Define a function to clean text data
def clean_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters and punctuation
    text = re.sub(r'[\u00BA\u00C4\u00C5\u00D4\u00E3\u00E8\u00EC\u00F2\u00F4\u00F9\u00FA\u0192]+', '', text)  # Remove specified characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove unnecessary whitespace
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters
    return text

# Clean the review texts
data['reviews.text'] = [clean_text(review) for review in data['reviews.text']]

# Map sentiment labels to integers
label_map = {'negative': 0, 'neutral': 1, 'positive': 2}
data['labels'] = [label_map[sentiment] for sentiment in data['sentiment']]

# Split the data into training, validation, and test sets
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    data['reviews.text'], data['labels'], test_size=0.2, random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42
)

# Create datasets.Dataset objects for train, validation, and test sets
train_data = {'reviews.text': train_texts, 'labels': train_labels}
val_data = {'reviews.text': val_texts, 'labels': val_labels}
test_data = {'reviews.text': test_texts, 'labels': test_labels}

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)
test_dataset = Dataset.from_dict(test_data)

# Combine datasets into a DatasetDict
dataset = DatasetDict({'train': train_dataset, 'validation': val_dataset, 'test': test_dataset})

In [ ]:
# Install verification for debugging reasons
!pip install transformers datasets sentencepiece gdown
!pip install evaluate
!pip install peft bitsandbytes accelerate
!pip install bitsandbytes accelerate transformers

In [ ]:
# Import model tools
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment", num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Tokenize function for mapping
def tokenize_function(examples):
    return tokenizer(
        examples['reviews.text'],
        padding='max_length',  # Ensure all sequences are padded to the same length
        truncation=True,      # Truncate sequences longer than the model's maximum input length
        max_length=128,        # Set a maximum length for the input sequences (adjust as needed)
        return_tensors="pt"  # Return PyTorch tensors
    )
# Tokenize datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/22661 [00:00<?, ? examples/s]

Map:   0%|          | 0/2833 [00:00<?, ? examples/s]

Map:   0%|          | 0/2833 [00:00<?, ? examples/s]

In [ ]:
# Convert tokenized datasets to TensorFlow datasets
def to_tf_dataset(ds, shuffle=True, batch_size=8):
    ds = ds.remove_columns(['reviews.text'])
    ds = ds.with_format("tensorflow")
    features = {x: ds[x] for x in tokenizer.model_input_names}
    tf_ds = tf.data.Dataset.from_tensor_slices((features, ds["labels"]))
    if shuffle:
        tf_ds = tf_ds.shuffle(buffer_size=len(ds))
    tf_ds = tf_ds.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return tf_ds

train_tf_dataset = to_tf_dataset(tokenized_datasets['train'])
val_tf_dataset = to_tf_dataset(tokenized_datasets['validation'], shuffle=False)
test_tf_dataset = to_tf_dataset(tokenized_datasets['test'], shuffle=False)

In [ ]:
# Install verification for debugging reasons
!pip install transformers[torch] accelerate -U

In [ ]:
# Ensure required packages are installed
!pip install transformers[torch] accelerate -U

from transformers import EarlyStoppingCallback, DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Define your create_optimizer function
def create_optimizer(init_lr, num_train_steps, num_warmup_steps):
    # Example implementation; replace with your actual optimizer and schedule creation
    from transformers import AdamW, get_linear_schedule_with_warmup
    optimizer = AdamW(model.parameters(), lr=init_lr)
    lr_schedule = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)
    return optimizer, lr_schedule

# Calculate the number of training steps
train_dataset_length = len(tokenized_datasets['train'])  # Ensure this is the correct dataset
num_train_steps = train_dataset_length * 5  # Adjust based on your specific needs

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/results',  # output directory
    eval_strategy="steps",  # evaluate each `logging_steps`
    learning_rate=2e-5,
    per_device_train_batch_size=48,  # batch size for training
    per_device_eval_batch_size=48,  # batch size for evaluation
    num_train_epochs=20,  # number of training epochs
    weight_decay=0.01,  # strength of weight decay
    logging_dir='/content/drive/MyDrive/logs',  # directory for storing logs
    load_best_model_at_end=True,  # load the best model when finished training (default metric is loss)
    metric_for_best_model="accuracy",
    save_total_limit=1,
    logging_steps=500,
    save_steps=500,  # Save and evaluate every 500 steps
    eval_steps=500,  # Evaluate every 500 steps to match saving frequency
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Use DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create optimizer and learning rate schedule
optimizer, lr_schedule = create_optimizer(init_lr=5e-5, num_train_steps=num_train_steps, num_warmup_steps=0)

# No conversion needed, use the original Hugging Face datasets
train_torch_dataset = tokenized_datasets['train']
val_torch_dataset = tokenized_datasets['validation']
test_torch_dataset = tokenized_datasets['test']

# Update the Trainer to use PyTorch datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_torch_dataset,  # Use PyTorch datasets
    eval_dataset=val_torch_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    optimizers=(optimizer, lr_schedule)
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Start the training
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.022700,0.142299,0.971055,0.971979,0.973495,0.971055
1000,0.017300,0.165467,0.975997,0.975854,0.975788,0.975997
1500,0.020300,0.163070,0.976350,0.976243,0.976500,0.976350
2000,0.022100,0.194294,0.972820,0.971790,0.971856,0.972820
2500,0.013400,0.170435,0.977409,0.977069,0.977231,0.977409
3000,0.018700,0.151749,0.975644,0.975818,0.976022,0.975644
3500,0.012300,0.153707,0.980586,0.980385,0.980257,0.980586
4000,0.019300,0.157389,0.975291,0.974951,0.974769,0.975291
4500,0.015500,0.162362,0.975644,0.975041,0.974680,0.975644
5000,0.013200,0.203138,0.971408,0.972194,0.973559,0.971408


TrainOutput(global_step=9460, training_loss=0.014155846670364477, metrics={'train_runtime': 4424.0141, 'train_samples_per_second': 102.445, 'train_steps_per_second': 2.138, 'total_flos': 2.981206579567104e+16, 'train_loss': 0.014155846670364477, 'epoch': 20.0})

In [ ]:
# Evaluate
results = trainer.evaluate()
print(results)

{'eval_loss': 0.15370726585388184, 'eval_accuracy': 0.9805859512883869, 'eval_f1': 0.9803853875813743, 'eval_precision': 0.9802570706540259, 'eval_recall': 0.9805859512883869, 'eval_runtime': 8.6502, 'eval_samples_per_second': 327.506, 'eval_steps_per_second': 6.936, 'epoch': 20.0}


In [ ]:
# Save the model
torch.save(model.state_dict(), "/content/drive/MyDrive/twitter-roberta-base-sentiment2/my_model_weights.h5")
trainer.save_model('/content/drive/MyDrive/twitter-roberta-base-sentiment2')
tokenizer.save_pretrained('/content/drive/MyDrive/twitter-roberta-base-sentiment2')

('/content/drive/MyDrive/twitter-roberta-base-sentiment2/tokenizer_config.json',
 '/content/drive/MyDrive/twitter-roberta-base-sentiment2/special_tokens_map.json',
 '/content/drive/MyDrive/twitter-roberta-base-sentiment2/vocab.json',
 '/content/drive/MyDrive/twitter-roberta-base-sentiment2/merges.txt',
 '/content/drive/MyDrive/twitter-roberta-base-sentiment2/added_tokens.json',
 '/content/drive/MyDrive/twitter-roberta-base-sentiment2/tokenizer.json')

In [ ]:
# Visualize the results

# Import evaluation tools
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Get the predictions
predictions = trainer.predict(test_torch_dataset)

# Convert the predictions to class labels
predicted_labels = np.argmax(predictions.predictions, axis=-1)

# Get the true labels
true_labels = test_torch_dataset['labels']

# Generate the classification report
report = classification_report(true_labels, predicted_labels)

# Print the classification report
print(report)

# Generate the confusion matrix
cf_matrix = confusion_matrix(true_labels, predicted_labels)

# Visualize the confusion matrix using seaborn
sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Test Data')
plt.show()

              precision    recall  f1-score   support

           0       0.94      0.88      0.91       155
           1       0.78      0.77      0.77       112
           2       0.99      0.99      0.99      2566

    accuracy                           0.98      2833
   macro avg       0.90      0.88      0.89      2833
weighted avg       0.98      0.98      0.98      2833



In [ ]:
# Attempt at Transfer Learning

# Ensure required packages are installed
!pip install transformers[torch] accelerate -U

from transformers import EarlyStoppingCallback, DataCollatorWithPadding, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import torch
import torch.nn as nn

# Load the previously trained model and tokenizer
pretrained_model_name = '/content/drive/MyDrive/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
pretrained_model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=3, ignore_mismatched_sizes=True)

# Load the base model for transfer learning
base_model_name = "bert-base-uncased"
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name, num_labels=3)

# Replace the classification head of BERT with that of the pretrained RoBERTa model
class CustomModel(nn.Module):
    def __init__(self, base_model, pretrained_model):
        super(CustomModel, self).__init__()
        self.base_model = base_model.bert  # Use BERT's backbone
        self.pretrained_head = pretrained_model.classifier  # Use RoBERTa's classifier
        # Initialize LayerNorm on the same device as the model
        self.layer_norm = nn.LayerNorm(768).to('cuda')
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs[0]  # Last hidden state
        pooled_output = hidden_state[:, 0, :]  # Extract the [CLS] token's embeddings
        pooled_output = self.layer_norm(pooled_output)  # Apply LayerNorm
        logits = self.pretrained_head(pooled_output)

        if labels is not None:
            loss = self.loss_fn(logits.view(-1, 3), labels.view(-1))
            return loss, logits
        return logits

# Initialize the custom model and move it to 'cuda' immediately
model = CustomModel(base_model, pretrained_model).to('cuda')

# Freeze the base model layers
for param in model.base_model.parameters():
    param.requires_grad = False

# Move the entire model to 'cuda' after initialization
model = CustomModel(base_model, pretrained_model).to('cuda')

# Tokenize function for mapping
def tokenize_function(examples):
    return tokenizer(
        examples['reviews.text'],
        padding='max_length',  # Ensure all sequences are padded to the same length
        truncation=True,      # Truncate sequences longer than the model's maximum input length
        max_length=128,        # Set a maximum length for the input sequences (adjust as needed)
        return_tensors="pt"  # Return PyTorch tensors
    )

# Tokenize datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define your create_optimizer function
def create_optimizer(init_lr, num_train_steps, num_warmup_steps):
    from transformers import AdamW, get_linear_schedule_with_warmup
    optimizer = AdamW(model.parameters(), lr=init_lr)
    lr_schedule = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)
    return optimizer, lr_schedule

# Calculate the number of training steps
train_dataset_length = len(tokenized_datasets['train'])
num_train_steps = train_dataset_length * 3  # Adjust based on your specific needs

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/results',  # output directory
    eval_strategy="steps",  # evaluate each `logging_steps`
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    num_train_epochs=20,  # number of training epochs
    weight_decay=0.01,  # strength of weight decay
    logging_dir='/content/drive/MyDrive/logs',  # directory for storing logs
    load_best_model_at_end=True,  # load the best model when finished training (default metric is loss)
    metric_for_best_model="accuracy",
    save_total_limit=1,
    logging_steps=500,
    save_steps=500,  # Save and evaluate every 500 steps
    eval_steps=500,  # Evaluate every 500 steps to match saving frequency
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Use DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create optimizer and learning rate schedule
optimizer, lr_schedule = create_optimizer(init_lr=5e-5, num_train_steps=num_train_steps, num_warmup_steps=0)

# Use PyTorch datasets
train_torch_dataset = tokenized_datasets['train']
val_torch_dataset = tokenized_datasets['validation']
test_torch_dataset = tokenized_datasets['test']

# Update the Trainer to use PyTorch datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_torch_dataset,  # Use PyTorch datasets
    eval_dataset=val_torch_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    optimizers=(optimizer, lr_schedule)
)

trainer.train()

results = trainer.evaluate()
print(results)

trainer.save_model('/content/drive/MyDrive/bert-base-finetuned')
tokenizer.save_pretrained('/content/drive/MyDrive/bert-base-finetuned')

from sklearn.metrics import classification_report

# Get the predictions
predictions = trainer.predict(test_torch_dataset)

# Convert the predictions to class labels
predicted_labels = np.argmax(predictions.predictions, axis=-1)

# Get the true labels
true_labels = test_torch_dataset['labels']

# Generate the classification report
report = classification_report(true_labels, predicted_labels)

# Print the classification report
print(report)

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Generate the confusion matrix
cf_matrix = confusion_matrix(true_labels, predicted_labels)

# Visualize the confusion matrix using seaborn
sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Test Data')
plt.show()

Some weights of the model checkpoint at /content/drive/MyDrive/twitter-roberta-base-sentiment were not used when initializing RobertaForSequenceClassification: ['base_model.embeddings.LayerNorm.bias', 'base_model.embeddings.LayerNorm.weight', 'base_model.embeddings.position_embeddings.weight', 'base_model.embeddings.token_type_embeddings.weight', 'base_model.embeddings.word_embeddings.weight', 'base_model.encoder.layer.0.attention.output.LayerNorm.bias', 'base_model.encoder.layer.0.attention.output.LayerNorm.weight', 'base_model.encoder.layer.0.attention.output.dense.bias', 'base_model.encoder.layer.0.attention.output.dense.weight', 'base_model.encoder.layer.0.attention.self.key.bias', 'base_model.encoder.layer.0.attention.self.key.weight', 'base_model.encoder.layer.0.attention.self.query.bias', 'base_model.encoder.layer.0.attention.self.query.weight', 'base_model.encoder.layer.0.attention.self.value.bias', 'base_model.encoder.layer.0.attention.self.value.weight', 'base_model.encoder.l

Map:   0%|          | 0/22661 [00:00<?, ? examples/s]

Map:   0%|          | 0/2833 [00:00<?, ? examples/s]

Map:   0%|          | 0/2833 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
